<a href="https://colab.research.google.com/github/YasiruMM/Medicine-Prediction-Grp-22/blob/Cardiovascular-Data-Cleaning/cardiovas_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Add configurations to repo
!git config --global user.email "yas.hm2mahanama@gmail.com"
!git config --global user.name "YasiruMM"

In [2]:
import pandas as pd
file_path= '/content/drive/My Drive/DSGP/Structured_CardiovascularFINAL_Data.xlsx'  # Replace with your actual file name
#Load the Excel file into a DataFrame
df = pd.read_excel(file_path)

# display head
df.head()


,Disease Category,Drug Category,Drug Name,Dosage,Retail Price,Purchase Price,Sales,Date
0,Cardiovascular,AMLODIP,AMLODAC 5MG,5MG,218.3,325.6,37.0,2.24
1,Cardiovascular,AMLODIP,AMLODAC 5MG,5MG,3100.7,2701.6,307.0,2.24
2,Cardiovascular,AMLODIP,AMLONG 10MG,10MG,59050.8,50814.4,1396.0,2.24
3,Cardiovascular,AMLODIP,AMLONG 2.5MG,2.5MG,136632.1,117638.4,6127.0,2.24
4,Cardiovascular,AMLODIP,AMLONG 5MG,5MG,512392.0,439915.5,16855.0,2.24


In [3]:
# Extract unique pairs of Drug Category and Drug Name
categoryNames_pair = df.groupby('Drug Category')['Drug Name'].unique().to_dict()
print(categoryNames_pair)



{'AMLODIP': array(['AMLODAC 5MG', 'AMLONG 10MG', 'AMLONG 2.5MG', 'AMLONG 5MG',
       'AMLOPRESS 5MG', 'AMLOSUN 5MG', 'STAMLO 5MG', 'STAMLO 10MG',
       'AMILODIPINE 10MG (CADLA)'], dtype=object), 'ASPIRIN': array(['ASPIN 100MG', 'ASPIRIN (SPC) 75MG', 'ASPRIN 75MG', 'ASTRIX 100MG',
       'CARDIPRIN 100MG', 'CAREPRIN 75MG', 'ECORIN 150MG', 'ECORIN 75MG',
       'ECOSPRIN 75MG', 'XSPRIN 75MG', 116.8, 'ASPIRIN 150MG', nan, 115.5,
       124.3, 127, 162.1, 111.6, 123.4], dtype=object), 'CLOPIDOGREL': array(['CLOPACT 75MG', 'CLOPID 75MG', 'CLOPIDOGREL(MEPRO) 75MG ',
       'CLOPIDOGREL 75MG', 'CLOPIL 75MG', 'CLOPILET 75MG',
       'CLOPITAB 75MG', 'CLOPIVAS 75MG', 'CLOPIVIC 75MG', 'DEPLATTE 75MG',
       'NOKLOT 75MG', 'PIDLET 75MG', 'PLAGERIN 75MG', 'PLATEX 75MG',
       'PLAVIX 75MG', 'PURIL 75MG', 'TROKAN 75MG', 496.2,
       'CLOPIDOGREL  75MG', 445.6, 517.3, 520.8,
       'CLOPIDOGREL (MEPRO)75MG', 'CLOPIDOGREL 75MG (MEPRO)', 487,
       'CLOPIDO 75MG', 'CLOPIDOGREL (MEPRO)75MG ', 46

In [4]:
# Identify most frequent (mode) 'Drug Category' and 'Drug Name'
most_frequent_category = df['Drug Category'].mode()[0]
most_frequent_drug_name = df['Drug Name'].mode()[0]


In [5]:
# Function to fill missing or invalid 'Drug Name' and 'Drug Category'
def fill_missing_values(row, category_to_names, most_frequent_category, most_frequent_drug_name):
    category = row['Drug Category']
    drug_name = row['Drug Name']

    # Handle missing 'Drug Category'
    if pd.isnull(category) or category not in category_to_names:
        row['Drug Category'] = most_frequent_category  # Fill with most frequent category

    # Handle missing or invalid 'Drug Name'
    if pd.isnull(drug_name) or isinstance(drug_name, (int, float)):
        row['Drug Name'] = most_frequent_drug_name  # Fill with most frequent drug name
    return row

# Apply the function to fill missing values
#df = df.apply(lambda row: fill_missing_values(row, categoryNames_pair, most_frequent_category, most_frequent_drug_name), axis=1)
# Show the updated dataframe
#print(df.head())


Handle Irrelevanr Dosage Values

In [6]:
# Finding the most frequent dosage for each drug name
most_frequent_dosages = df.groupby('Drug Category')['Dosage'].agg(lambda x: x.mode()[0]).to_dict()
print(most_frequent_dosages)


{'AMLODIP': '5MG', 'ASPIRIN': '75MG', 'CLOPIDOGREL': '75MG', 'ENALA': '5MG', 'LOSARTAN': '50MG'}


In [7]:
# Function to fill missing & irrelevant 'Dosages'
def fill_invalid_dosage(row, dosage_map):
    dosage = row['Dosage']
    category=row['Drug Category']

    # Check if dosage is missing or invalid (you can add specific criteria for invalid values)
    if pd.isnull(dosage) or dosage== '':
        # If invalid or missing, use the most frequent dosage for the drug
        row['Dosage'] = dosage_map.get(category,None)  #use most fequent dosage

    return row

In [8]:
# Apply the function to the DataFrame
#df = df.apply(lambda row: fill_invalid_dosage(row, most_frequent_dosages), axis=1)

#print(df.head())

Handle Numerical Fields

In [9]:
# retrieve medians of specific numerics under each drug category.
median_values = df.groupby('Drug Category').agg({
    'Retail Price': 'median',
    'Purchase Price': 'median',
    'Sales': 'median'
}).to_dict()
print(median_values)

{'Retail Price': {'AMLODIP': 49871.8, 'ASPIRIN': 3522.6, 'CLOPIDOGREL': 13470.0, 'ENALA': 8210.95, 'LOSARTAN': 15256.8}, 'Purchase Price': {'AMLODIP': 56521.85, 'ASPIRIN': 3797.6, 'CLOPIDOGREL': 15685.9, 'ENALA': 13589.6, 'LOSARTAN': 14347.9}, 'Sales': {'AMLODIP': 1550.0, 'ASPIRIN': 668.0, 'CLOPIDOGREL': 898.0, 'ENALA': 1523.0, 'LOSARTAN': 1048.0}}


In [10]:
# Step 2: Function to handle missing or irrelevant values
def fill_numeric_values(row, median_dict):
    category = row['Drug Category']

    # Handle Retail Price
    if pd.isnull(row['Retail Price']) or row['Retail Price'] <= 0 or not isinstance(row['Retail Price'], (int, float)):  # Null or invalid values handling
        row['Retail Price'] = median_dict['Retail Price'].get(category, df['Retail Price'].median())

    # Handle Purchase Price
    if pd.isnull(row['Purchase Price']) or row['Purchase Price'] <= 0 or not isinstance(row['Purchase Price'], (int, float)):  # Null or invalid values handling
        row['Purchase Price'] = median_dict['Purchase Price'].get(category, df['Purchase Price'].median())

    # Handle Sales
    if pd.isnull(row['Sales']) or row['Sales'] <= 0 or not isinstance(row['Sales'], (int, float)):  # Null or invalid values handling
        row['Sales'] = median_dict['Sales'].get(category, df['Sales'].median())

    return row

In [11]:
# Step 3: Apply the function to the dataframe
#df = df.apply(lambda row: fill_numeric_values(row, median_values), axis=1)
#print(df.head())

# AUDIT REPORTS for ALL UPDATES

In [12]:
def transformations(df):
  # trans 1: Handle drug category and drug names values
  df = df.apply(lambda row: fill_missing_values(row, categoryNames_pair, most_frequent_category, most_frequent_drug_name), axis=1)
  # trans 2: Handle Dosage Values
  df = df.apply(lambda row: fill_invalid_dosage(row, most_frequent_dosages), axis=1)
  # trans 3: Handle Numericals
  df = df.apply(lambda row: fill_numeric_values(row, median_values), axis=1)
  return df

In [13]:
def audit_repo(name_func,df,*args,**kwargs):
  origin=df.copy() # backup to original dataset
  df_update=name_func(df,*args,**kwargs) # applying each transformation

  changes = (origin != df_update)  # Compare the original and updated dataframes to identify changed cells
  changed_col=changes.any(axis=0) # Identify columns with changes
  changed_rows=changes.any(axis=1) # Identify rows with changes

  # Create a summary report
  change_report = {
        "changed_col": list(changed_col[changed_col].index),  # List of changed columns
        "changed_rows": df.loc[changed_rows],  # Rows with changes
    }

  return df_update, change_report


In [14]:
updation,report=audit_repo(transformations,df)
print(report)
print("Columns with changes:", report["changed_col"])
print("Rows with changes:", report["changed_rows"])


{'changed_col': ['Drug Name', 'Dosage', 'Retail Price', 'Purchase Price', 'Sales'], 'changed_rows':     Disease Category Drug Category      Drug Name Dosage  Retail Price  \
15    Cardiovascular       AMLODIP    STAMLO 10MG   10MG         948.0   
23    Cardiovascular       AMLODIP    STAMLO 10MG   10MG         368.0   
31    Cardiovascular       AMLODIP    STAMLO 10MG   10MG         666.0   
41    Cardiovascular       AMLODIP    STAMLO 10MG   10MG        1144.0   
53    Cardiovascular       AMLODIP    STAMLO 10MG   10MG         760.0   
62    Cardiovascular       AMLODIP    STAMLO 10MG   10MG         683.0   
72    Cardiovascular       AMLODIP    STAMLO 10MG   10MG         817.0   
81    Cardiovascular       AMLODIP    STAMLO 10MG   10MG        1099.0   
95    Cardiovascular       ASPIRIN          116.8    NaN        1153.0   
113   Cardiovascular       ASPIRIN            NaN    NaN       90457.6   
129   Cardiovascular       ASPIRIN          115.5    NaN        1252.0   
144   Cardio

In [15]:
from google.colab import files
# Save the processed data to a CSV file in Colab's local environment
output_csv_path = '/content/Updated2_CardioDATA.csv'
updation.to_csv(output_csv_path, index=False)
files.download(output_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Group by 'Drug Name' and 'Date', and aggregate
df_grouped = df.groupby(['Drug Name', 'Date'], as_index=False).agg({
    'Disease Category': 'first',       # Take the first value for Disease Category (if consistent)
    'Drug Category': 'first', # Take the first value for Drug Category (if consistent)
    'Dosage': 'first',                 # Take the first value for Dosage (if consistent)
    'Retail Price': 'first',           # Take the first value for Retail Price (if consistent)
    'Purchase Price': 'first',         # Take the first value for Purchase Price (if consistent)
    'Sales': 'sum'                     # Sum the Sales for duplicates
                        # Take the first value for Date (if consistent)
})

# Display the grouped result
print(df_grouped)


      Drug Name   Date Disease Category Drug Category Dosage  Retail Price  \
0          43.9   9.24   Cardiovascular         ENALA   None         261.0   
1          45.9   2.24   Cardiovascular         ENALA   None         435.0   
2          45.9   3.24   Cardiovascular         ENALA   None         363.0   
3          55.7   6.24   Cardiovascular         ENALA   None         587.0   
4          55.8   7.24   Cardiovascular         ENALA   None         361.0   
..          ...    ...              ...           ...    ...           ...   
729  ZAART 50MG   6.24   Cardiovascular      LOSARTAN   50MG     1568739.6   
730  ZAART 50MG   7.24   Cardiovascular      LOSARTAN   50MG     1609213.2   
731  ZAART 50MG   8.24   Cardiovascular      LOSARTAN   50MG     1592281.2   
732  ZAART 50MG   9.24   Cardiovascular      LOSARTAN   50MG     1527307.2   
733  ZAART 50MG  10.24   Cardiovascular      LOSARTAN   50MG     1556071.2   

     Purchase Price    Sales  
0           12812.0      0.0  
1

In [ ]:
from google.colab import files
# Save the processed data to a CSV file in Colab's local environment
output_csv_path = '/content/cardiorep_Handle.csv'
df_grouped.to_csv(output_csv_path, index=False)
# Download the CSV file to your local machine
files.download(output_csv_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>